In [1]:
#from IPython.display import display, HTML
#display(HTML("<style>div.output_scroll { height: 100em; }</style>"))
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import itertools
from itertools import chain
import random

def defvars(label, n):
    return list(var(label + '_%d' % i) for i in range(n))

def defpoly_from_basis(label, basis):
    coeffs = defvars(label,len(basis))
    poly = sum(c*x for c,x in zip(coeffs,basis))
    return (coeffs,poly)

def defpoly(label, d):
    return defpoly_from_basis(label, list(x**i for i in range(d+1)))

def concat(lists):
    return list(itertools.chain.from_iterable(lists))

# For given d, let P = prod_{j=0}^{d-1} (X-j), then this function
# returns the coefficient of P corresponding to X^i for given i
def stirling(d,i):
    if i == 0:
        return 0
    stirlingR = PolynomialRing(SR,'stir')
    p = prod(stirlingR.0-j for j in range(d))
    print(p)
    print(p.coefficients(stirlingR.0)[i-1])
    
def v_coeff(i,j,x):
    return binomial(i,j)*(x**(i-j))

#print(v_coeff(1,1,var('bla')))
#print(concat([[5,6],[6],[2,1,1]]))

def subs_vec(vec,subsmap):
    return vector([e.subs(subsmap) for e in vec])

def subs_mat(mat,subsmap):
    return Matrix([[e.subs(subsmap) for e in row] for row in mat])

# Vertically stacks two vectors
def stack(vec1,vec2):
    return vector(list(vec1) + list(vec2))

In [3]:
def print_coeff(eq,mon):
    show(mon)
    if mon == 1:
        print(eq.constant_coefficient())
    else:
        print(eq.monomial_coefficient(mon))
    print("--------------------------")

In [4]:
n = 5             # number of equations
nx = 4             # number of instance elements
m = 7             # number of witness elements
nh = 4            # number of random bases

hs = [var('H%d' % i) for i in range(nh)]


# # For uBlu
# d = 3
# n = 3*d+4
# nx = 3 + 2*d
# m = 2*d+8

# hs = [var('H0'),var('H1')] + list(var('W%d' % (i+1)) for i in range(d)) # Some hash-to-curve elements
# nh = len(hs)

def generate_random_language():
    mat = [[0 for j in range(m)] for i in range(n)]
    for i in range(n):
        for j in range(m):
            choice = random.random()
            if choice < 0.6:
                mat[i][j] = lambda inst: 0
            elif choice < 0.95:
                mat[i][j] = lambda inst: 1
            else:
                ix = random.randint(0, nx-1)
                mat[i][j] = lambda inst: inst[ix]

    vec = [0 for i in range(n-nx)]
    for i in range(n-nx):
        # Each element is randomly either 0, 1, or inst[i]
        choice = random.random()
        if choice < 0.33:
            vec[i] = lambda inst: 0
        elif choice < 0.66:
            vec[i] = lambda inst: 1
        else:
            ix = random.randint(0, nx-1)
            vec[i] = lambda inst: inst[ix]
    
    def random_M(inst):
        mat_instantiated = [[mat[i][j](inst) for j in range(m)] for i in range(n)]
        return Matrix(mat_instantiated)

    def random_theta(inst):
        vec_instantiated = [vec[i](inst) for i in range(n-nx)]
        return vector(vec_instantiated)
        
    return random_M, random_theta


# Generate matrices automatically
# M, theta = generate_random_language()

# # Manual generation:
# def M(inst):
#     mat = [[0 for j in range(m)] for i in range(n)]
#     # mat[0][0] = 1         # x1 = G^w1
#     # mat[1][1] = hs[0]     # x2 = H^w2
#     # mat[2][1] = inst[0]   # x3 = x1^w2 = G^{w1 w2}
    
#     # mat[3][2] = 1         # x3 = G^{w3}
    
#     # mat[4][0] = inst[1]   # x3^{-1} = x2^w1 (H^-1 G^-1)^w3
#     # mat[4][2] = (-1 - hs[0])

#     # Powers of 1 witness plus 2nd witness mixed in, no blinders
#     mat[0][0] = 1         # x1 = G^w1 
#     mat[1][0] = inst[0]   # x2 = x1^w1 = G^{w1^2}
#     mat[2][1] = inst[1]   # x3 = x2^w2 = G^{w1^2 w2}
        
#     return Matrix(mat)

# def theta(inst):
#     vec = [0 for i in range(n-nx)]

#     # vec[0] = inst[2]
#     # vec[1] = -inst[2]

#     return vector(vec)


# 1 -> (*-1) -1 -> (*-1) 1

# X -> (*5) Y -> (*5) X

# X^7 - 4 * X^3 - 2 = 0

# X^4 - 1   => would capture roots of unity of deg 4 if they exist

# 7, 13,    (X - 7)(X - 13) = 0
# X          X + alpha = Y mod p, Y + alpha = X mod p

#  X         A X + B = Y,   A Y + B = X
#            (A - 1) (X + Y) + 2 B = 0   mod p ?
#                             2 ((p-1)/2)
#            (X+Y)^{-1} (X+Y)  - 1  = 0

def Mtheta(inst):

    mat = [[0 for j in range(m)] for i in range(n)]
    vec = [0 for i in range(n - nx)]

    
    ############ Diffie Hellman w/o additives
    
    # mat[0][0] = 1
    # mat[0][3] = hs[0]

    # mat[1][1] = 1
    # mat[1][4] = hs[1]

    # mat[2][2] = 1
    # mat[2][5] = hs[2]

    # mat[3][1] = inst[0]
    # mat[3][2] = -1
    # mat[3][6] = -hs[0]
    
    # vec[0] = 0

    ########## forcing upd 1 to w0 and w1
    
    # # x1 = G^w1 H^w3
    # mat[0][0] = 1    
    # mat[0][2] = hs[0]
    
    # mat[1][1] = 1    # x2 = G^w2

    # # x3 = (x1)^w2 (x2)^w2 = G^{w1 w2 + w2^2} H^w3w2
    # mat[2][1] = inst[0] + inst[1]

    # # # # x4 = x3^{w1 + w2} = G^{(w1^2 + w2^2) (w1 + w2)}
    # # mat[3][0] = inst[2] #+ hs[0]
    # # mat[3][1] = inst[2] #+ hs[1]

    # # 1 = inst[0]^w1 H^w4 = G^{w1^2} H^w2w1 H^w4 => w4 = -w3w1
    # mat[3][0] = inst[0]
    # mat[3][3] = hs[0]
    # vec[0] = 1


    ##########  mult <-> add upd

    # x0 = G^w0 H^w2
    mat[0][0] = 1    
    mat[0][2] = hs[0]

    # x1 = G^w1
    mat[1][1] = 1    
    mat[1][4] = hs[1]

    # x2 = (x0 + 1)^w1 H1^w4 = G^{w0 w1 + w1} H0^w2w1 H1^w4
    mat[2][1] = inst[0] + 1
    mat[2][5] = hs[2]

    # to constrain w1 from changing at all
    mat[3][1] = inst[0] + inst[1]
    mat[3][6] = hs[3]
    
    # 1 = inst[0]^w0 H^w3 = G^{w0^2} H^w0w1 H^w3      
    # => w4 = -w3w1  
    # => w0^2 = 1
    mat[4][0] = inst[0]
    mat[4][3] = hs[0]
    vec[0] = 1


    ##########
    
    #mat[2][1] = 2 * inst[0]

    #mat[1][0] = inst[0]
    #mat[4][2] = hs[0]
    
    #mat[3][2] = hs[0]

    #mat[4][2] = 1
    #mat[2][1] = 2 * inst[0]
    #mat[2][2] = hs[0]
    
    # # x1 = G^w1 H0^w5
    # mat[0][0] = 1
    # mat[0][4] = hs[0]
    
    # # x2 = G^w2 H1^w6
    # mat[1][1] = 1
    # mat[1][5] = hs[1]

    # # x3 = (G^{w1 + w2}) H2^w7
    # mat[2][2] = 1
    # mat[2][6] = hs[2]

    # # x4 = x3^(w1+w2) H3^w9
    # mat[3][3] = 1
    # mat[3][8] = hs[3]
    

    # # x1+x2 = G^w3 H0^w5 H0^w6      ~=     w3 = w1 + w2
    # mat[4][2] = 1
    # mat[4][4] = hs[0]
    # mat[4][5] = hs[1]

    # # 1 = x3^w3 G^{-w4} H2^{-w8}     ~= w3 = (w1+w2)^2, w8 = whatever
    # mat[5][2] = inst[2]
    # mat[5][3] = -1
    # mat[5][7] = -hs[2]

    return (Matrix(mat), vector(vec))


def M(inst):
    mat,vec = Mtheta(inst)
    return mat


def theta(inst):   
    mat,vec = Mtheta(inst)
    return vec
    

    

############ (?) BC
### The following language has 4 transformations, and transformation # 2 is not generically blinding-compatible: 
### It is only blinding compatible if r9: r46 + r47, where r47 is a new Ta associated variable, and r46 is Tx variable.
### @Volhovm: this is OK up to variable replacement: r46+r47 -> alpha, r47 -> beta, then r46 = alpha - beta.

# def M(inst):
#     mat = [[0 for j in range(m)] for i in range(n)]
# 
#     mat[0][0] = 1         # x1 = G^w1
#     mat[1][1] = inst[0]   # x2 = x1^w1 = G^{w1 w2}
#     mat[2][2] = 1         # x1 * x2 = G^w3
#         
#     return Matrix(mat)
# 
# def theta(inst):
#     vec = [0 for i in range(n-nx)]
# 
#     vec[0] = inst[0] + inst[1]         # x1 * x3
# 
#     return vector(vec)
# 
#  solution # 1:
# {Tx2_0: r8, Tx2_1: r11*r8, Tx1_0_0: r9, Tx1_0_1: r10, Tx1_1_0: r11*r9, Tx1_1_1: r10*r11, Tw2_0: r8, Tw2_1: r11, Tw2_2: (r11 + 1)*r8, Tw1_0_0: -r10 + r9, Tw1_0_1: 0, Tw1_0_2: r10, Tw1_1_0: 0, Tw1_1_1: 0, Tw1_1_2: 0, Tw1_2_0: -r10*(r11 + 1) + (r11 + 1)*r9, Tw1_2_1: 0, Tw1_2_2: r10*(r11 + 1)}
#  Tw1:
# [                    -r10 + r9                             0                           r10]
# [                            0                             0                             0]
# [-r10*(r11 + 1) + (r11 + 1)*r9                             0                 r10*(r11 + 1)]
#  Tw2:
# (r8, r11, (r11 + 1)*r8)
#  Tx1:
# [     r9     r10]
# [ r11*r9 r10*r11]
#  Tx2:
# (r8, r11*r8)



######### (?) BC
# This one has a single transformation, but two distinct BC variants of that, not one unified BC variant. Why?
#    mat[0][0] = 1         # x1 = G^w1
#    mat[1][0] = inst[0]   # x2 = x1^w1 = G^{w1^2}
#    mat[2][1] = 1         # x2 = G^w2       => w2 = w1^2
#    mat[3][2] = 1         # x2 = G^w3       => w3 = w1^2

# BC (attempts)
    # Powers of 1 witness plus 2nd witness mixed in, no blinders
    #mat[0][0] = 1         # x1 = G^w1 
    #mat[1][0] = inst[0]   # x2 = x1^w1 = G^{w1^2}
    #mat[2][1] = inst[1]   # x3 = x2^w2 = G^{w1^2 w2}

    # Powers of 1 witness
    #mat[0][0] = 1         # x1 = G^w1
    #mat[1][0] = inst[0]   # x2 = x1^w1 = G^{w1^2}
    #mat[2][0] = inst[1]   # x3 = x2^w1 = G^{w1^3}

    # Powers of 1 witness plus some blinders?
    #mat[0][0] = 1         # x1 = G^w1 H^w3
    #mat[0][2] = hs[0]
    #mat[1][0] = inst[0]   # x2 = x1^w1 = G^{w1^2} H^{w3 w1}
    #mat[2][1] = inst[1]   # x3 = x2^w2 H^w4 = G^{w1^2 w2} H^{w3 w1 w2 + w4}
    #mat[2][3] = hs[0]


# Tx1 * x + Tx2
# Tx1 * (x || H) + Tx2


# transformation ???
#
# x1' = x1 * G^alpha
# x2' = x2 * x1^{2 alpha} * G^{alpha^2}
# x3' = x3 * 
#  (w1 + alpha)^2 w2 = w1 w2 + 2 alpha w1 w2 + alpha^2 w2


xs = [var('x_%d' % i) for i in range(nx)]
ws = [var('w_%d' % i) for i in range(m)]
x = vector(xs)
w = vector(ws)

# z variables are akin to w, but used for modelling "any witness" for blinding compatibility
zs = [var('z_%d' % i) for i in range(m)]
z = vector(zs)

assert(n == M(xs).nrows())
assert(m == M(xs).ncols())

print("M")
print(M(xs))
print("theta")
print(theta(xs))
    
inst_toprint = M(xs) * vector(ws)
for i in range(nx):
    print(xs[i], "=", inst_toprint[i])

print("thetas:")
for i in range(n-nx):
    print(theta(xs)[i], "=", inst_toprint[nx+i])

def find_coeff_of_var(expr, var):
    return expr.coefficient(var, 1)

def find_constant_coeff(expr):
    return expr.subs({v : 0 for v in expr.variables()})

# Matrix m as a tensor
m_tens = [[[find_coeff_of_var(M(xs)[i][j], xs[t]) for t in range(nx)] for j in range(m)] for i in range(n)]
m_tens_h = [[[find_coeff_of_var(M(xs)[i][j], hs[t]) for t in range(len(hs))] for j in range(m)] for i in range(n)]
m_tens_const = [[find_constant_coeff(M(xs)[i][j]) for j in range(m)] for i in range(n)]

for t in range(nx):
    print("main inst t: ", t)
    print(Matrix([[m_tens[i][j][t] for j in range(m)] for i in range(n)]))

for t in range(nh):
    print("h t: ", t)
    print(Matrix([[m_tens_h[i][j][t] for j in range(m)] for i in range(n)]))

print("const: ")
print(Matrix(m_tens_const))

M
[        1         0        H0         0         0         0         0]
[        0         1         0         0        H1         0         0]
[        0   x_0 + 1         0         0         0        H2         0]
[        0 x_0 + x_1         0         0         0         0        H3]
[      x_0         0         0        H0         0         0         0]
theta
(1)
x_0 = H0*w_2 + w_0
x_1 = H1*w_4 + w_1
x_2 = H2*w_5 + w_1*(x_0 + 1)
x_3 = H3*w_6 + w_1*(x_0 + x_1)
thetas:
1 = H0*w_3 + w_0*x_0
main inst t:  0
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 1 0 0 0 0 0]
[1 0 0 0 0 0 0]
main inst t:  1
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 0 0 0 0 0]
main inst t:  2
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
main inst t:  3
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
h t:  0
[0 0 1 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 1 0 0 0]
h t:  1
[0 0 0 0 0 0 0]
[0 0 

In [5]:
Txs = [[var('Tx1_%d_%d' % (i, j)) for j in range(nx + nh + 1)] for i in range(nx)]
Tws = [[var('Tw1_%d_%d' % (i,j)) for j in range(m + 1)] for i in range(m)]


#for i in range(m):
#    Tw1s[i][i] = 1
 
#us = [var('u0'),var('u1')]

#Tw2s[0] = var('u0') # w0 + u0
#Tw2s[1] = var('u1') # w1 + u1

#T_reduce_map = {var('U_α'): 0, var('w_α'): 0}
T_reduce_map = {}


# for i in range(nx):
#     for j in range(nx):
#         if i != j:
#             T_reduce_map[Txs[i][j]] = 0
#         # else:
#         #     T_reduce_map[Tx1s[i][j]] = 1

# for i in range(m):
#     for j in range(m):
#         if i != j:
#             T_reduce_map[Tws[i][j]] = 0
#         # else:
#         #     T_reduce_map[Tw1s[i][j]] = 1

Tx = Matrix(subs_mat(Txs,T_reduce_map))
Tw = Matrix(subs_mat(Tws,T_reduce_map))


print(Tw)
print("--------------------")
print(Tx)
print("--------------------")
#print(us)

# Backups
Txs_orig = Txs
Tws_orig = Tws

Tx_orig = Tx
Tw_orig = Tw

ts_map = {}

[Tw1_0_0 Tw1_0_1 Tw1_0_2 Tw1_0_3 Tw1_0_4 Tw1_0_5 Tw1_0_6 Tw1_0_7]
[Tw1_1_0 Tw1_1_1 Tw1_1_2 Tw1_1_3 Tw1_1_4 Tw1_1_5 Tw1_1_6 Tw1_1_7]
[Tw1_2_0 Tw1_2_1 Tw1_2_2 Tw1_2_3 Tw1_2_4 Tw1_2_5 Tw1_2_6 Tw1_2_7]
[Tw1_3_0 Tw1_3_1 Tw1_3_2 Tw1_3_3 Tw1_3_4 Tw1_3_5 Tw1_3_6 Tw1_3_7]
[Tw1_4_0 Tw1_4_1 Tw1_4_2 Tw1_4_3 Tw1_4_4 Tw1_4_5 Tw1_4_6 Tw1_4_7]
[Tw1_5_0 Tw1_5_1 Tw1_5_2 Tw1_5_3 Tw1_5_4 Tw1_5_5 Tw1_5_6 Tw1_5_7]
[Tw1_6_0 Tw1_6_1 Tw1_6_2 Tw1_6_3 Tw1_6_4 Tw1_6_5 Tw1_6_6 Tw1_6_7]
--------------------
[Tx1_0_0 Tx1_0_1 Tx1_0_2 Tx1_0_3 Tx1_0_4 Tx1_0_5 Tx1_0_6 Tx1_0_7 Tx1_0_8]
[Tx1_1_0 Tx1_1_1 Tx1_1_2 Tx1_1_3 Tx1_1_4 Tx1_1_5 Tx1_1_6 Tx1_1_7 Tx1_1_8]
[Tx1_2_0 Tx1_2_1 Tx1_2_2 Tx1_2_3 Tx1_2_4 Tx1_2_5 Tx1_2_6 Tx1_2_7 Tx1_2_8]
[Tx1_3_0 Tx1_3_1 Tx1_3_2 Tx1_3_3 Tx1_3_4 Tx1_3_5 Tx1_3_6 Tx1_3_7 Tx1_3_8]
--------------------


In [6]:
# Optional: perform a substitution mapping for instance/witness variables
#tsmod = {t_α: 0, t_rα: 0}
tsmod = {}
x2 = vector([x[i] * (1 if x[i].subs(ts_map).subs(tsmod).full_simplify() != 0 else 0) for i in range(nx)])
w2 = vector([w[i] * (1 if w[i].subs(ts_map).subs(tsmod).full_simplify() != 0 else 0) for i in range(m)])
x2w2_map = {ws[i]:w2[i] for i in range(m)} | {xs[i]:x2[i] for i in range(len(xs))}
print(x2w2_map)
print("-------")

Tx = subs_mat(Tx,x2w2_map)
Tw = subs_mat(Tw,x2w2_map)


# Our main equation
eq_basic = stack(x2,theta(x2)) - M(x2) * w2

print("Main equation, by instance vector component 0..n:")
for i in range(n):
    print(" ", i, ": ", eq_basic[i].full_simplify())
print("-------")

# Update equation
#eq_u = Tx1 * (M(x2) * w2) + Tx2 - (M(Tx1 * x2 + Tx2) * (Tw1 * w2 + Tw2))
#x_upd_2 = Tx1_top * (M(x) * w) + Tx2_top

# Temporarily disable additive components: I'm looking to see identity multiplicative matrices.
x_upd = Tx * stack(stack(x, hs), [1])
w_upd = Tw * stack(w, [1])
# x_upd = Tx1 * x 
# w_upd = Tw1 * w 

eq_u = stack(x_upd, theta(x_upd)) - M(x_upd) * w_upd

#eq_u = stack(Tx1_top * (M(x) * w) + Tx2_top, theta(Tx1_top * (M(x) * w) + Tx2_top)) - (M(Tx1 * stack(x,theta(x)) + Tx2) * (Tw1 * w + Tw2))
print("Update equation, by instance vector component 0..n:")
for (e,i) in zip(eq_u,range(n)):
    print(" ", i, ": ", e.full_simplify())

{w_0: w_0, w_1: w_1, w_2: w_2, w_3: w_3, w_4: w_4, w_5: w_5, w_6: w_6, x_0: x_0, x_1: x_1, x_2: x_2, x_3: x_3}
-------
Main equation, by instance vector component 0..n:
  0 :  -H0*w_2 - w_0 + x_0
  1 :  -H1*w_4 - w_1 + x_1
  2 :  -H2*w_5 - w_1*x_0 - w_1 + x_2
  3 :  -H3*w_6 - w_1*x_0 - w_1*x_1 + x_3
  4 :  -H0*w_3 - w_0*x_0 + 1
-------
Update equation, by instance vector component 0..n:
  0 :  -H0*Tw1_2_7 + H0*Tx1_0_4 + H1*Tx1_0_5 + H2*Tx1_0_6 + H3*Tx1_0_7 - (H0*Tw1_2_0 + Tw1_0_0)*w_0 - (H0*Tw1_2_1 + Tw1_0_1)*w_1 - (H0*Tw1_2_2 + Tw1_0_2)*w_2 - (H0*Tw1_2_3 + Tw1_0_3)*w_3 - (H0*Tw1_2_4 + Tw1_0_4)*w_4 - (H0*Tw1_2_5 + Tw1_0_5)*w_5 - (H0*Tw1_2_6 + Tw1_0_6)*w_6 + Tx1_0_0*x_0 + Tx1_0_1*x_1 + Tx1_0_2*x_2 + Tx1_0_3*x_3 - Tw1_0_7 + Tx1_0_8
  1 :  -H1*Tw1_4_7 + H0*Tx1_1_4 + H1*Tx1_1_5 + H2*Tx1_1_6 + H3*Tx1_1_7 - (H1*Tw1_4_0 + Tw1_1_0)*w_0 - (H1*Tw1_4_1 + Tw1_1_1)*w_1 - (H1*Tw1_4_2 + Tw1_1_2)*w_2 - (H1*Tw1_4_3 + Tw1_1_3)*w_3 - (H1*Tw1_4_4 + Tw1_1_4)*w_4 - (H1*Tw1_4_5 + Tw1_1_5)*w_5 - (H1*Tw1_4_6 +

In [7]:
# Solve for dependencies between witnesses. It seems only reasonable to AGM-like solve over the independent basis 
# of trapdoors, and if w_i are dependent, we should first express them in terms of a basis.

x2_flat = [elem for elem in x2]
w2_flat = [elem for elem in w2]
hs_flat = [elem for elem in hs]
print(x2_flat + w2_flat + hs_flat)


sols = solve(list(eq_basic) + [w[3] + w[0] * w[2], w[0] * w[0] - 1], hs_flat + w2_flat + x2_flat, solution_dict=True)
# sols = solve(list(eq_basic) + [w2_flat[0]^2-1], x2_flat + w2_flat + hs_flat, solution_dict=True)
#sols = solve(list(eq_basic) + constraints, w2_flat + x2_flat + hs_flat, solution_dict=True)
print(sols)

# some solutions will assume e.g. hs[0] = 0, which is undesirable.
# We only want to keep solutions where hs[i] is free variable
# we only keep these solutions, and remove hs keys from it, replacing these free variables
# by hs[i] variables directly
def filter_sols_with_hs_free(sols):
    retset = []
    for sol in sols:
        if any([len(sol[hs_i].variables()) != 1 for hs_i in hs]):
            continue

        if len(set(list([sol[hs_i].variables() for hs_i in hs]))) != len(hs):
            continue

        modsol = {k: v.subs({sol[hs_i]: hs_i for hs_i in hs}) for k,v in sol.items()}
        for hs_i in hs:
            modsol.pop(hs_i)
        
        retset.append(modsol)
    
    return retset

sols = filter_sols_with_hs_free(sols)
print("filtered len: ", len(sols))
print("filtered: ", sols)
print("\n")

chosen_sol = sols[0]

# We need solutions that leave hs as fully free variables.

var('rr_0')
var('rr_1')
var('rr_2')
var('rr_4')
var('rr_5')
var('rr_6')
param_solution = { x_0: rr_0 + H0 * rr_2, 
                   x_1: rr_1 + H1 * rr_4, 
                   x_2: rr_0 * rr_1 + rr_1 + H0 * rr_2 * rr_1 + H2 * rr_5, 
                   x_3: rr_0 * rr_1 + rr_1 * rr_1 + H0 * rr_2 * rr_1 + H1 * rr_4 * rr_1 + H3 * rr_6, 
                   w_0: rr_0, 
                   w_1: rr_1, 
                   w_2: rr_2, 
                   w_3: rr_0 * (-rr_2),
                   w_4: rr_4,
                   w_5: rr_5,
                   w_6: rr_6 }
param_values = [ {rr_0: 1}, {rr_0: -1} ]
print("Parameterized solution:", param_solution)
print("Parameter values:", param_values)

print("Does this solution work? main eq by instance vector component 0..n: (should be zeroes)")
for i in range(n):
    print(" ", i, ": ", eq_basic[i].subs(param_solution).full_simplify())

chosen_sol = param_solution


def get_basis_from_sols(sol):
    # Assuming sols[0] is your solution dictionary
    params = set()  # find all parameters (like r4)
    for value in sol.values():
        params.update(value.variables())

    return list(params)

eq_basic_params = [x for x in get_basis_from_sols(chosen_sol) if x not in hs]
print("eq_basic_params: ", eq_basic_params)

# eq_generic_params = [rr_0, rr_1, rr_2]
# eq_constrained_params = [rr_0]

#eq_u_via_params = [eq.subs(chosen_sol).full_simplify() for eq in eq_u]
eq_u_via_params = [eq.subs(chosen_sol).full_simplify().subs(rr_0^2 == 1) for eq in eq_u]
print("eq_u: ")
for (e,i) in zip(eq_u,range(n)):
    print(" ", i, ": ", e)
print("eq_u_via_params: ", )
for (e,i) in zip(eq_u_via_params,range(n)):
    print(" ", i, ": ", e)

x_via_eq_basic_params = vector([elem.subs(chosen_sol) for elem in x2])
print(x_via_eq_basic_params)

[x_0, x_1, x_2, x_3, w_0, w_1, w_2, w_3, w_4, w_5, w_6, H0, H1, H2, H3]
[{H0: r1, H1: r2, H2: r3, H3: r4, w_0: 1, w_1: r5, w_2: r6, w_3: -r6, w_4: r7, w_5: r8, w_6: r9, x_0: r1*r6 + 1, x_1: r2*r7 + r5, x_2: r1*r5*r6 + r3*r8 + 2*r5, x_3: r1*r5*r6 + r2*r5*r7 + r5^2 + r4*r9 + r5}, {H0: r10, H1: r11, H2: r12, H3: r13, w_0: -1, w_1: r14, w_2: r15, w_3: r15, w_4: r16, w_5: r17, w_6: r18, x_0: r10*r15 - 1, x_1: r11*r16 + r14, x_2: r10*r14*r15 + r12*r17, x_3: r10*r14*r15 + r11*r14*r16 + r14^2 + r13*r18 - r14}]
filtered len:  2
filtered:  [{w_0: 1, w_1: r5, w_2: r6, w_3: -r6, w_4: r7, w_5: r8, w_6: r9, x_0: H0*r6 + 1, x_1: H1*r7 + r5, x_2: H0*r5*r6 + H2*r8 + 2*r5, x_3: H0*r5*r6 + H1*r5*r7 + r5^2 + H3*r9 + r5}, {w_0: -1, w_1: r14, w_2: r15, w_3: r15, w_4: r16, w_5: r17, w_6: r18, x_0: H0*r15 - 1, x_1: H1*r16 + r14, x_2: H0*r14*r15 + H2*r17, x_3: H0*r14*r15 + H1*r14*r16 + r14^2 + H3*r18 - r14}]


Parameterized solution: {x_0: H0*rr_2 + rr_0, x_1: H1*rr_4 + rr_1, x_2: H0*rr_1*rr_2 + rr_0*rr_1 + H2

In [8]:
# Helper functions for solving equations AGM-style
# - Symbolic form is how equations are defined without a ring
# - "poly" or ring form is polynomials over a ring. only in this form we can extract monomials.

# Takes a ring and a list of variables, and re-assigns these variables to a ring.
# Returns:
#   - to_sym_map is a map that maps a ring variable to the original symbolic base variable
#   - ret is a list of mapped variables in a ring
def reassign_vars(R,varlists):
    ret = []
    totsum = 0
    to_sym_map = {} 
    for base_vars in varlists:
        ring_vars = [R.gen(i) for i in range(totsum,totsum+len(base_vars))]
        ret.append(ring_vars)
        to_sym_map = to_sym_map | {ring_vars[i]:base_vars[i] for i in range(len(base_vars))} 
        totsum += len(base_vars)
    return (to_sym_map,ret)

# Ringvars are the "Trapdoors" over which we want to extract AGM sub-equations
ringvars = [hs, xs, ws, zs, eq_basic_params]
R = PolynomialRing(SR, concat(ringvars))
#R.inject_variables()
(to_sym_map,[poly_hs,poly_xs,poly_ws,poly_zs, poly_eq_basic_params]) = reassign_vars(R,ringvars)

print(ringvars)

# Convert a polynomial from ring form to the symbolic form
def poly_to_sym(poly):
    print(poly.subs(to_sym_map))
    return poly.subs(to_sym_map)

# Convert a polynomial from symbolic form to ring form
def sym_to_poly(poly):
    from sage.symbolic.expression_conversions import polynomial
    return polynomial(poly, ring=R)

[[H0, H1, H2, H3], [x_0, x_1, x_2, x_3], [w_0, w_1, w_2, w_3, w_4, w_5, w_6], [z_0, z_1, z_2, z_3, z_4, z_5, z_6], [rr_6, rr_2, rr_1, rr_0, rr_5, rr_4]]


In [ ]:
# AGM-solve for witness/instance transformations (Tx, Tw) but NOT Ta
# *TODO* rn it's not AGM because adversary can't see the instance
# Assume the most generic form of Tw
# Model Tx as an algebraic sum of instance elements and fixed elements like H
# Take update equations, line by line, and extract sub-equations by monomial coefficients in trapdoor vars
# Trapdoor vars (secrets) are ... witness elements I think, plus DLOGs of hash-to-curve els

Tx_vars_flat = [x for sublist in Txs for x in sublist]
Tw_vars_flat = [x for sublist in Tws for x in sublist]

print(Tw_vars_flat + Tx_vars_flat)


# Transform equation to symbolic form
#eq_u_updated = [sym_to_poly(eq.subs(basic_sol)) for eq in eq_u]
eq_u_updated = [sym_to_poly(eq) for eq in eq_u_via_params]
#eq_u_updated = [sym_to_poly(eq.subs({k:basic_sol[k] for k in xs})) for eq in eq_u]
print("updated eqs: ", eq_u_updated)
print("-------")

all_monomials = list(dict.fromkeys([mon for eq in eq_u_updated for mon in eq.monomials()]))
print("all monomials: ", all_monomials)
print("-------")
    
coeff_eqs = [eq.monomial_coefficient(mon) for eq in eq_u_updated for mon in eq.monomials() ]
print("coeff_eqs: ", coeff_eqs)
for (e,i) in zip(coeff_eqs,range(0,len(coeff_eqs))):
    print(" ", i, ": ", e)
print("-------")

#assert(false)

def print_solutions(solutions):
    print("------------ ", len(solutions)," FILTERED (generic) solutions")
    for sol_i,sol in enumerate(solutions):
        print("============\n solution # %d:" % sol_i)
        if any([all(map(lambda x: x == 0, [Tx.subs(sol)[i][j] for j in range(nx + nh + 1)])) for i in range(nx)]):
            print("TRIVIAL SOL, INST ROW 0")
            continue
        
        print(sol)
    
        print(" Tw1:")
        print(Tw.subs(sol))
    

        print(" Tx1:")
        print(Tx.subs(sol))

def iterative_solve(equations, variables):
    # Progressive solving
    partial_solutions = {}
    remaining_eqs = equations[:]
    
    # Solve in batches
    batch_size = 10
    for i in range(0, len(equations), batch_size):
        print("\n-------- processing next batch")
        batch = equations[i:i+batch_size]
        try:
            sols = solve(batch, variables, solution_dict=True, algorithm='sympy')
            if sols:
                print(sols)
                partial_solutions.update(sols[0])
                assert(false)
                print_solutions(partial_solutions)
                # Substitute known values into remaining equations
                remaining_eqs = [eq.subs(partial_solutions) for eq in remaining_eqs[batch_size:]]
            else:
                print("solution was not found, whoops")
                assert(false)
        except:
            continue

    return partial_solutions

# sols = solve(coeff_eqs, Tw_vars_flat + Tx_vars_flat, solution_dict=True, algorithm='sympy')
#sols = solve(coeff_eqs, Tw_vars_flat + Tx_vars_flat, solution_dict=True, algorithm='fricas')
sols = iterative_solve(coeff_eqs, Tw_vars_flat + Tx_vars_flat)


print("------------ ", len(sols)," solutions")


def is_special_case_of(sol1, sol2):
    """Check if sol1 is a special case of sol2"""
    # Create a system of equations comparing the solutions
    equations = []
    variables = set()
    
    for var in sol2:
        eq = sol1[var] == sol2[var]
        equations.append(eq)
        variables.update(sol2[var].variables())

    # Try to solve for the variables in sol2 that would make sol1 = sol2
    result = solve(equations, list(variables))
    return len(result) > 0  # If there's a solution, sol1 is a special case

def filter_generic_solutions(solutions):
    filtered = []
    for i, sol1 in enumerate(solutions):
        is_most_generic = True
        for j, sol2 in enumerate(solutions):
            if i != j:
                # Try substituting sol1 into sol2 to check if sol1 is more specific
                try:
                    if is_special_case_of(sol1,sol2):  # if substitution works, sol1 is less generic
                        is_most_generic = False
                        break
                except:
                    continue
        if is_most_generic:
            filtered.append(sol1)
    return filtered

sols = filter_generic_solutions(sols)


#TODO Test solutions for validity

print_solutions(sols)

[Tw1_0_0, Tw1_0_1, Tw1_0_2, Tw1_0_3, Tw1_0_4, Tw1_0_5, Tw1_0_6, Tw1_0_7, Tw1_1_0, Tw1_1_1, Tw1_1_2, Tw1_1_3, Tw1_1_4, Tw1_1_5, Tw1_1_6, Tw1_1_7, Tw1_2_0, Tw1_2_1, Tw1_2_2, Tw1_2_3, Tw1_2_4, Tw1_2_5, Tw1_2_6, Tw1_2_7, Tw1_3_0, Tw1_3_1, Tw1_3_2, Tw1_3_3, Tw1_3_4, Tw1_3_5, Tw1_3_6, Tw1_3_7, Tw1_4_0, Tw1_4_1, Tw1_4_2, Tw1_4_3, Tw1_4_4, Tw1_4_5, Tw1_4_6, Tw1_4_7, Tw1_5_0, Tw1_5_1, Tw1_5_2, Tw1_5_3, Tw1_5_4, Tw1_5_5, Tw1_5_6, Tw1_5_7, Tw1_6_0, Tw1_6_1, Tw1_6_2, Tw1_6_3, Tw1_6_4, Tw1_6_5, Tw1_6_6, Tw1_6_7, Tx1_0_0, Tx1_0_1, Tx1_0_2, Tx1_0_3, Tx1_0_4, Tx1_0_5, Tx1_0_6, Tx1_0_7, Tx1_0_8, Tx1_1_0, Tx1_1_1, Tx1_1_2, Tx1_1_3, Tx1_1_4, Tx1_1_5, Tx1_1_6, Tx1_1_7, Tx1_1_8, Tx1_2_0, Tx1_2_1, Tx1_2_2, Tx1_2_3, Tx1_2_4, Tx1_2_5, Tx1_2_6, Tx1_2_7, Tx1_2_8, Tx1_3_0, Tx1_3_1, Tx1_3_2, Tx1_3_3, Tx1_3_4, Tx1_3_5, Tx1_3_6, Tx1_3_7, Tx1_3_8]
updated eqs:  [(Tx1_0_2 + Tx1_0_3)*H0*rr_2*rr_1 + Tw1_2_3*H0*rr_2*rr_0 + Tx1_0_3*H1*rr_1*rr_4 + (-Tw1_2_6)*H0*rr_6 + Tx1_0_3*H3*rr_6 + (-Tw1_2_2 + Tx1_0_0)*H0*rr_2 + (-Tw1